## Environment Setting

In [ ]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install dominate==2.4.0
!pip install detectron2==0.1.2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

## Clone the file from Gihub

In [ ]:
!git clone https://github.com/ericsujw/InstColorization.git

In [ ]:
cd InstColorization/

/content/Inst


## Start Colorization

## Download the Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#the path from my Google Drive where everything regarding the project will be saved
path = '/content/drive/My Drive/InstColorization'

In [ ]:
!python train.py

In [ ]:
#paath to test images
input_dir = path + '/example'

In [ ]:
from os.path import join, isfile, isdir
from os import listdir
import os

In [ ]:
#path to train images
train_data = path + '/train_data/train2017'

In [ ]:
!pip install visdom

In [ ]:
import visdom

In [ ]:
from torch.nn import init
import torch.nn as nn

In [ ]:
class Siggraph(nn.Module):
    def __init__(self, norm_layer=nn.BatchNorm2d):
        super(Siggraph, self).__init__()
        use_bias = True
        # Conv1
        model1=[nn.Conv2d(4, 64, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model1+=[nn.ReLU(True),]
        model1+=[nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model1+=[norm_layer(64),]
        model1+=[nn.ReLU(True),]
        
        # Conv2
        model2=[nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model2+=[nn.ReLU(True),]
        model2+=[nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model2+=[norm_layer(128),]
        model2+=[nn.ReLU(True),]

        # Conv3
        model3=[nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model3+=[norm_layer(256),]
        model3+=[nn.ReLU(True),]

        # Conv4
        model4=[nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=use_bias),]
        model4+=[norm_layer(512),]
        model4+=[nn.ReLU(True),]
        

        # Conv5
        model5=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model5+=[norm_layer(512),]
        model5+=[nn.ReLU(True),]
        


        model_aux6=[nn.Conv2d(512, 256, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model_aux6+=[norm_layer(256),]
        model_aux6+=[nn.ReLU(True),]
        

        model_aux7=[nn.Conv2d(256, 128, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model_aux7+=[norm_layer(128),]
        model_aux7+=[nn.ReLU(True),]
        

        model_aux8=[nn.Conv2d(128, 64, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model_aux8+=[norm_layer(64),]
        model_aux8+=[nn.ReLU(True),]
        

        model_aux9=[nn.Conv2d(64, 32, kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias),]
        model_aux9+=[norm_layer(32),]
        model_aux9+=[nn.ReLU(True),]
        

        model_aux10=[nn.Linear(in_features = 32768, out_features = 10),]

        self.model1 = nn.Sequential(*model1)
        self.model2 = nn.Sequential(*model2)
        self.model3 = nn.Sequential(*model3)
        self.model4 = nn.Sequential(*model4)
        self.model5 = nn.Sequential(*model5)
        self.model_aux6 = nn.Sequential(*model_aux6)
        self.model_aux7 = nn.Sequential(*model_aux7)
        self.model_aux8 = nn.Sequential(*model_aux8)
        self.model_aux9 = nn.Sequential(*model_aux9)
        self.model_aux10 = nn.Sequential(*model_aux10)


    def forward(self, input_A, input_B, mask_B):
        conv1_2 = self.model1(torch.cat((input_A,input_B,mask_B),dim=1))
        conv2_2 = self.model2(conv1_2[:,:,::2,::2])
        conv3_3 = self.model3(conv2_2[:,:,::2,::2])
        conv4_3 = self.model4(conv3_3[:,:,::2,::2])
        conv5_3 = self.model5(conv4_3)
        conv6 = self.model_aux6(conv5_3)
        conv7 = self.model_aux7(conv6)
        conv8 = self.model_aux8(conv7)
        conv9 = self.model_aux9(conv8)
        conv9 = conv9.view(25, -1)
        conv10 = self.model_aux10(conv9)
        
        return conv10


def init_net(net, init_type='xavier'):
    net = torch.nn.DataParallel(net)
    init_weights(net, init_type)
    return net


def init_weights(net, init_type='xavier', gain=0.02):
    def init_func(m):
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            if init_type == 'normal':
                init.normal_(m.weight.data, 0.0, gain)
            elif init_type == 'xavier':
                init.xavier_normal_(m.weight.data, gain=gain)
            elif init_type == 'kaiming':
                init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
            elif init_type == 'orthogonal':
                init.orthogonal_(m.weight.data, gain=gain)
            else:
                raise NotImplementedError('initialization method [%s] is not implemented' % init_type)
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)
        elif classname.find('BatchNorm2d') != -1:
            init.normal_(m.weight.data, 1.0, gain)
            init.constant_(m.bias.data, 0.0)

    print('initialize network with %s' % init_type)
    net.apply(init_func)


In [ ]:
#path_full = '/content/drive/My Drive/InstColorization/checkpoints/instance/coco_mask/latest_net_G.pth'
#path_instance = '/content/drive/My Drive/InstColorization/checkpoints/full/coco_mask/latest_net_G.pth'
path_instance = '/content/drive/My Drive/check_full/new/latest_net_G.pth'
#path_instance = '/content/drive/My Drive/checkpoints2/coco_mask/latest_net_G.pth'

In [ ]:
#saves data about the objects detected with Detectron2
!python inference_bbox.py

In [ ]:
import torch

In [ ]:
class MyModelInst():
    def __init__(self):
        #self.netG = init_net(Siggraph(), 'xavier')
        self.netG = Siggraph()
        #self.netG = Siggraph()
        #self.netG.init_net(self.netG, 'xavier')
        self.loss = torch.nn.CrossEntropyLoss()
        self.mask_cent = 0.5

    # set requies_grad=Fasle to avoid computation
    def set_requires_grad(self, nets, requires_grad=False):
        if not isinstance(nets, list):
            nets = [nets]
        for net in nets:
            if net is not None:
                for param in net.parameters():
                    param.requires_grad = requires_grad


    def optimize_parameters(self, target):
        pred = self.forward()
        #acc = torch.sum(torch.argmax(target, dim=1)==torch.argmax(pred.to(torch.device('cpu')), dim=1))
        #pred.to(torch.device('cuda:0'))
        #acc = torch.sum(torch.argmax(target.to(torch.device('cuda')), dim=1)==torch.argmax(pred, dim=1))
        acc = torch.sum(target == torch.argmax(pred, dim=1))
        loss = self.loss(pred, target)
        
        loss.backward()
        self.optimizer_G.step()
        self.optimizer_G.zero_grad()

        return loss, acc

    def forward(self):
        pred = self.netG(self.real_A, self.hint_B, self.mask_B)
        return pred

    def set_input(self, input):
        AtoB = self.opt.which_direction == 'AtoB'
        self.real_A = input['A' if AtoB else 'B']
        self.real_B = input['B' if AtoB else 'A']
        self.hint_B = input['hint_B']
        
        self.mask_B = input['mask_B']
        self.mask_B_nc = self.mask_B + self.mask_cent

        self.real_B_enc = util.encode_ab_ind(self.real_B[:, :, ::4, ::4], self.opt)

    def save_networks(self, which_epoch):
          name = 'G'
          save_filename = '%s_net_%s.pth' % (which_epoch, name)
          save_path = os.path.join('/content/drive/My Drive/check_full/new', save_filename)    #path to checkpoints for the classification-full model
          net = self.netG
          torch.save(net.cpu().state_dict(), save_path)


In [ ]:
loaded_state_dict = torch.load(path_instance)


my_model_inst = MyModelInst()

own_state = my_model_inst.netG.state_dict()
for name, param in loaded_state_dict.items():
    if name not in own_state:
        continue
    param = param.data
    own_state[name].copy_(param)

my_model_inst.netG.load_state_dict(own_state)

for idx, param in enumerate(my_model_inst.netG.parameters()):
  if(idx<51):
    param.requires_grad = False

my_model_inst.optimizer_G = torch.optim.Adam(my_model_inst.netG.parameters(),
                                                lr=0.0001, betas=(0.9, 0.999))

In [ ]:
import torch.utils.data as Data

In [ ]:
import image_util
import importlib
import util
importlib.reload(image_util)
importlib.reload(util)
from util import util
from image_util import *
from os import listdir
from os.path import isfile, join
from random import sample

import numpy as np

In [ ]:
class Training_Instance_Dataset(Data.Dataset):
    '''
    Training on COCOStuff dataset. [train2017.zip]
    
    Download the training set from https://github.com/nightrome/cocostuff

    Make sure you've predicted all the images' bounding boxes using inference_bbox.py

    It would be better if you can filter out the images which don't have any box.
    '''
    def __init__(self):
        self.PRED_BBOX_DIR = '/content/drive/My Drive/train_bbox'
        self.IMAGE_DIR = '/content/drive/My Drive/train'
        self.IMAGE_ID_LIST = [f for f in listdir(self.IMAGE_DIR) if isfile(join(self.IMAGE_DIR, f))]
        self.transforms = transforms.Compose([
            transforms.Resize((256, 256), interpolation=2),
            transforms.ToTensor()
        ])
        self.labels_train = torch.zeros(994, dtype = torch.int64)
        self.labels_train[:95] = 1
        self.labels_train[95:195] = 9
        self.labels_train[195:294] = 3
        self.labels_train[294:394] = 5
        self.labels_train[394:494] = 6
        self.labels_train[494:594] = 4
        self.labels_train[594:694] = 2
        self.labels_train[694:794] = 7
        self.labels_train[794:894] = 0
        self.labels_train[794:994] = 8

    def __getitem__(self, index):
        pred_info_path = join(self.PRED_BBOX_DIR, self.IMAGE_ID_LIST[index].split('.')[0] + '.npz')
        output_image_path = join(self.IMAGE_DIR, self.IMAGE_ID_LIST[index])
        pred_bbox = gen_maskrcnn_bbox_fromPred(pred_info_path)

        rgb_img, gray_img = gen_gray_color_pil(output_image_path)
        #print('here')
        index_list = range(len(pred_bbox))
        if index_list !=range(0, 0):
          index_list = sample(index_list, 1)
          startx, starty, endx, endy = pred_bbox[index_list[0]]
          output = {}
          output['rgb_img'] = self.transforms(rgb_img.crop((startx, starty, endx, endy)))
          output['gray_img'] = self.transforms(gray_img.crop((startx, starty, endx, endy)))
        else:
          output = {}
          output['rgb_img'] = self.transforms(rgb_img)
          output['gray_img'] = self.transforms(gray_img)
        output['label'] = self.labels_train[index]
        return output

    def __len__(self):
        return len(self.IMAGE_ID_LIST)

In [ ]:
from torchvision import transforms

In [ ]:
dataset = Training_Instance_Dataset()
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=25, shuffle=True, num_workers=8)

In [ ]:
from tqdm import trange, tqdm

In [ ]:
import sre_compile

In [ ]:
class Opt():
  def __init__(self):
      self.l_norm = 100.
      self.l_cent = 50.
      self.ab_norm = 110.
      self.ab_max = 110. 
      self.ab_quant = 10.
      self.mask_cent = .5
      self.batch_size = 25
      self.loadSize = 256
      self.fineSize = 256
      self.input_nc = 1
      self.output_nc = 2
      self.ngf = 64
      self.ndf = 64
      self.which_model_netD = 'basic'
      self.which_model_netG = 'siggraph'
      self.n_layers_D = 3
      self.dataset_mode = 'aligned'
      self.which_direction ='AtoB'
      self.nThreads = 4
      #self.chekpoints_dir = '/content/drive/My Drive/InstColorization/checkpoints/instance/coco_mask/latest_net_G.pth'
      self.norm = 'batch'
      self.sample_p = 1.0
      self.sample_Ps = [1, 2, 3, 4, 5, 6, 7, 8, 9, ]
      self.aspect_ratio = 1.0
      self.A = sre_compile.SRE_FLAG_ASCII

In [ ]:
#train instance model
batch_size=25
accuracy = 0
total_steps = 0
opt = Opt()
my_model_inst.opt = opt
niter = 0
for_plot = []
#my_model_inst.set_requires_grad(my_model_inst.netG, True)
for epoch in trange(0, 100, desc='epoch', dynamic_ncols=True):
      epoch_iter = 0
      accuracy_total = 0
      for data_raw in tqdm(dataset_loader, desc='batch', dynamic_ncols=True, leave=False):
            total_steps += batch_size
            epoch_iter += batch_size

            data_raw['rgb_img'] = [data_raw['rgb_img']]
            data_raw['gray_img'] = [data_raw['gray_img']]

            input_data = util.get_colorization_data(data_raw['gray_img'], opt, p=1.0, ab_thresh=0)
            gt_data = util.get_colorization_data(data_raw['rgb_img'], opt, p=1.0, ab_thresh=10.0)
            if gt_data is None:
                continue
            if(gt_data['B'].shape[0] < batch_size):
                continue
            input_data['B'] = gt_data['B']
            input_data['hint_B'] = gt_data['hint_B']
            input_data['mask_B'] = gt_data['mask_B']
            niter += 1
            my_model_inst.set_input(input_data)
            loss, acc = my_model_inst.optimize_parameters(data_raw['label'])
            accuracy_total += float(acc)/float(batch_size)
            accuracy = float(acc)/float(batch_size)
            for_plot.append(100.0*accuracy)
            if epoch % 5 == 0:
                print("Epoch = {0} Loss = {1} Accuracy = {2}".format(epoch, loss, 100*accuracy))
            my_model_inst.save_networks('latest')
            my_model_inst.save_networks(epoch)
            #my_model_inst.update_learning_rate()
      acc = 100*accuracy_total/niter
      print("Total accuracy: {0}".format(acc))


In [ ]:
!python inference_bbox.py

 30% 30/99 [00:08<00:20,  3.37it/s]libpng warning: iCCP: known incorrect sRGB profile
100% 99/99 [00:33<00:00,  2.91it/s]


In [ ]:
import importlib
import Inst

In [ ]:
class Testing_Instance_Dataset(Data.Dataset):
    '''
    Training on COCOStuff dataset. [train2017.zip]
    
    Download the training set from https://github.com/nightrome/cocostuff

    Make sure you've predicted all the images' bounding boxes using inference_bbox.py

    It would be better if you can filter out the images which don't have any box.
    '''
    def __init__(self):
        self.PRED_BBOX_DIR = '/content/drive/My Drive/train_bbox/test_bbox'
        self.IMAGE_DIR = '/content/drive/My Drive/train_bbox/test'
        self.IMAGE_ID_LIST = [f for f in listdir(self.IMAGE_DIR) if isfile(join(self.IMAGE_DIR, f))]
        self.transforms = transforms.Compose([
            transforms.Resize((256, 256), interpolation=2),
            transforms.ToTensor()
        ])

        self.labels_train = torch.zeros(100, dtype = torch.int64)
        self.labels_train[:10] = 1
        self.labels_train[10:20] = 9
        self.labels_train[20:30] = 3
        self.labels_train[30:40] = 5
        self.labels_train[40:50] = 6
        self.labels_train[50:60] = 4
        self.labels_train[60:70] = 2
        self.labels_train[70:80] = 7
        self.labels_train[80:90] = 0
        self.labels_train[90:100] = 8

    def __getitem__(self, index):
        pred_info_path = join(self.PRED_BBOX_DIR, self.IMAGE_ID_LIST[index].split('.')[0] + '.npz')
        output_image_path = join(self.IMAGE_DIR, self.IMAGE_ID_LIST[index])
        pred_bbox = gen_maskrcnn_bbox_fromPred(pred_info_path)

        rgb_img, gray_img = gen_gray_color_pil(output_image_path)
        index_list = range(len(pred_bbox))
        if index_list !=range(0, 0):
          index_list = sample(index_list, 1)
          startx, starty, endx, endy = pred_bbox[index_list[0]]
          output = {}
          output['rgb_img'] = self.transforms(rgb_img.crop((startx, starty, endx, endy)))
          output['gray_img'] = self.transforms(gray_img.crop((startx, starty, endx, endy)))
        else:
          output = {}
          output['rgb_img'] = self.transforms(rgb_img)
          output['gray_img'] = self.transforms(gray_img)
        output['label'] = self.labels_train[index]
        return output

    def __len__(self):
        return len(self.IMAGE_ID_LIST)

In [ ]:
dataset = Testing_Instance_Dataset()
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=25, shuffle=True, num_workers=8)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_res(x, y, color='blue', title = ''):
    plt.plot(x, y, '-', color=color)
    plt.title(title)

In [ ]:
path_loss = '/content/drive/My Drive/InstColorization/checkpoints/plot/loss_log.txt'    #path to text log loss file